In [ ]:
from __future__ import unicode_literals

from sklearn import metrics
import sklearn.metrics as sm
from sklearn import manifold
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.impute import SimpleImputer
from sklearn import preprocessing

import math
import random
import time

from hazm import *
import numpy as np
import pandas as pd
from collections import Counter
import csv, re, pickle

from colorama import Back, Fore, Style
import time

import matplotlib.pyplot as plt
from matplotlib import pyplot as plt

import pyclustering as pyclus

import seaborn as sns
sns.set()

In [ ]:
data = pd.read_excel("dataset.xlsx")
reviews = data['question']
# rate = data['Score']
labels = list(data['subject'])

In [ ]:


RE_USELESS = r'[^\w]'  # remove useless characters
RE_DIGIT = r"^\d+\s|\s\d+\s|\s\d+$"  # remove digits
RE_SPACE = r'\s+'  # remove space
RE_EMAILS = r'[\w\.-]+@[\w\.-]+'
RE_URLS = r'http\S+'
RE_WWW = r'www\S+'


def clean_all_save(document, save_file_path):
    """
    this function generate raw persian text, it remove non-persian character
    and all numbers and symbols
    :param document:
    :param save_file_path:
    :return:
    """
    with open(save_file_path, 'w') as output:
        for sentence in document:
            sentence = clean_sentence(sentence)
            output.write(sentence + '\n')
    return None


def clean_all(document, doc_pattern=r'<TEXT>(.*?)</TEXT>'):
    """
    clean text like hamshahri, irBlogs, and other Treck format
    :param document:
    :param doc_pattern:
    :return:
    """
    clean = ''
    document = re.findall(doc_pattern, document, re.DOTALL)
    for sentence in document:
        sentence = clean_sentence(sentence)
        clean += ' \n' + sentence
    return clean


def clean_sentence(sentence):
    sentence = re.sub(r'[^\u0621-\u06ff]', ' ', sentence)
    sentence = arToPersianChar(sentence)
    sentence = arToPersianNumb(sentence)
    sentence = faToEnglishNumb(sentence)
    sentence = re.sub(r'[a-zA-Z]', ' ', sentence)
    sentence = re.sub(r'[0-9]', ' ', sentence)
    sentence = re.sub(RE_WWW, r' ', sentence)
    sentence = re.sub(RE_URLS, r' ', sentence)
    sentence = re.sub(RE_EMAILS, r' ', sentence)
    sentence = re.sub(RE_USELESS, r' ', sentence)
    sentence = re.sub(RE_DIGIT, r' ', sentence)
    sentence = re.sub(RE_SPACE, r' ', sentence)
    return sentence


def arToPersianNumb(number):
    dic = {
        '١': '۱',
        '٢': '۲',
        '٣': '۳',
        '٤': '۴',
        '٥': '۵',
        '٦': '۶',
        '٧': '۷',
        '٨': '۸',
        '٩': '۹',
        '٠': '۰',
    }
    return multiple_replace(dic, number)


def arToPersianChar(userInput):
    dic = {
        'ك': 'ک',
        'دِ': 'د',
        'بِ': 'ب',
        'زِ': 'ز',
        'ذِ': 'ذ',
        'شِ': 'ش',
        'سِ': 'س',
        'ى': 'ی',
        'ي': 'ی'
    }
    return multiple_replace(dic, userInput)


def faToEnglishNumb(number):
    dic = {
        '۰': '0',
        '۱': '1',
        '۲': '2',
        '۳': '3',
        '۴': '4',
        '۵': '5',
        '۶': '6',
        '۷': '7',
        '۸': '8',
        '۹': '9',
    }
    return multiple_replace(dic, number)


def multiple_replace(dic, text):
    pattern = "|".join(map(re.escape, dic.keys()))
    return re.sub(pattern, lambda m: dic[m.group()], str(text))

In [ ]:
def clean_all(document):
    clean = ''
    for sentence in document:
        sentence = clean_sentence(sentence)
        clean += sentence
    return clean

In [ ]:
j = k = i = 0
reviews1 = []
labels1 = []
# labels1 = list(labels.copy())
normalizer = Normalizer()
for review in reviews:
    sentences = sent_tokenize(normalizer.normalize(clean_all(review)))
    reviews1.extend(sentences)
    for j in range(len(sentences)):
        labels1.insert(i + k, labels[i])
        k += 1
    i += 1

In [ ]:
print(len(reviews1),len(labels1))

In [ ]:
#cleaning dataset
words=[]
all_text = ''
# stemmer = Stemmer()
for t in range (len(reviews1)):
    text = reviews1[t]
    text = text.replace('\u200c',' ')
    text = text.replace('\u200f',' ')
    text = re.sub(r'[^a-zA-Z0-9آ-ی۰-۹ ]', ' ', text)
    all_text += text
    all_text += ' '
    wordsInText = text.split()
    for word in wordsInText:
#         word = stemmer.stem(word)
        if word != ' ' or word != '':
            words.append(word)
len(words)

In [ ]:
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

with open("mySavedDict.txt", "wb") as myFile:
    pickle.dump(vocab_to_int, myFile)

'''
with open("mySavedDict.txt", "rb") as myFile:
    myNewPulledInDictionary = pickle.load(myFile)
'''

In [ ]:
reviews_ints = []
for each in reviews1:
    #print (each)
    each = each.replace('\u200c',' ')
    each = each.replace('\u200f',' ')
    each = re.sub(r'[^a-zA-Z0-9آ-ی۰-۹ ]', ' ', each)
    reviews_ints.append([vocab_to_int[word] for word in each.split()])


review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))


In [ ]:
mi = 15000
su = ma = 0
i = 0
low = []
for each in reviews_ints:
    if len(each) == 2 or len(each) == 1:
        low.append(i)
    if len(each) <= mi:
#         print(each,i,len(each))
        mi = len(each)
    if len(each) > ma:
        ma = len(each)
    su += len(each)
    i += 1
print('min lenght: '+str(mi),' and max lenght: '+str(ma),' and mean lenght: '+str(su/len(reviews_ints)))



In [ ]:
reviews_ints22 = reviews_ints.copy()
for i in range(len(low)):
    print(reviews_ints22.pop(low[len(low)- i -1]),low[len(low)- i -1])
len(reviews_ints22)


In [ ]:
reviews_ints = reviews_ints22.copy()

In [ ]:
seq_len = 30
X = np.zeros((len(reviews_ints), seq_len), dtype=int)

for i, row in enumerate(reviews_ints):
#     print (i , row)
#     print (i )
#     print ('****')
    X[i, -len(row):] = np.array(row)[:seq_len]


In [ ]:


NORMALIZATION = True


def readVars(config_file):
    config = configparser.ConfigParser()
    config.read(config_file)
    budget = int(config.get("vars", "budget"))
    kmax = int(config.get("vars", "kmax"))  # Maximum number of Clusters
    numOfInd = int(config.get("vars", "numOfInd"))  # number of individual
    Ps = float(config.get("vars", "Ps"))
    Pm = float(config.get("vars", "Pm"))
    Pc = float(config.get("vars", "Pc"))

    return budget, kmax, Ps, Pm, Pc, numOfInd


# minmax normalization
def minmax(data):
    normData = data
    data = data.astype(float)
    normData = normData.astype(float)
    for i in range(0, data.shape[1]):
        tmp = data.iloc[:, i]
        # max of each column
        maxElement = np.amax(tmp)
        # min of each column
        minElement = np.amin(tmp)

        # norm_dat.shape[0] : size of row
        for j in range(0, normData.shape[0]):
            normData[i][j] = float(
                data[i][j] - minElement) / (maxElement - minElement)

    normData.to_csv('result/norm_data.csv', index=None, header=None)
    return normData
data = pd.DataFrame(X)
data = minmax(data)  # normalize


In [ ]:
X=data.iloc[:,:].values

dataPoints=X.shape[0]
features=X.shape[1]

In [ ]:


# #Preprocessing
# ordinalEncoder=preprocessing.OrdinalEncoder()
# ordinalList=[ordinalEncoder for i in range(dataPoints)]
# for feature in range(X.shape[1]):
#     o=preprocessing.OrdinalEncoder()
#     missingValueImputer=SimpleImputer(missing_values=np.nan,strategy="constant", fill_value=0)
#     X[:,feature]=missingValueImputer.fit_transform(X[:,feature].reshape(-1,1)).reshape(-1,)
#     X[:,feature]=o.fit_transform(X[:,feature].reshape(-1,1)).reshape(-1,)
#     ordinalList[feature]=o;

# missingValueImputer=SimpleImputer(missing_values=np.nan,strategy="constant", fill_value=0)
# X=missingValueImputer.fit_transform(X)


In [ ]:
X

In [ ]:

##apply WOA-kmeans clustering

numberOfCluster=3
numberOfWhale=10
iterations=40
# dataPoints=X.shape[0]
# features=X.shape[1];
#intialise

centresOfwhale=np.zeros((numberOfWhale,numberOfCluster,features))


for whale in range(numberOfWhale):
    for cluster in range(numberOfCluster):
        for feature in range(features):
            centresOfwhale[whale,cluster,feature]=float(random.randint(np.min(X[:,feature]),np.max(X[:,feature])))
bestWhale=0
for iteration in range(iterations):
    print(iteration)
    #dataPointsInCluster=[[[] for cluster in range(numberOfCluster)] for whale in range(numberOfWhale)]
    dataPointsInCluster=np.zeros((numberOfWhale,numberOfCluster))
    bestWhale=0
    bestDist=float("inf")
    startTime=time.time()
    for whale in range(numberOfWhale):
        dist=0.00
        clus1=np.zeros((features))
        clus2=np.zeros((features))
        for dataPoint in range(dataPoints):
            bestEuclidianDist=float("inf")
            bestCluster=0
            for cluster in range(numberOfCluster):
                euclidDist=np.linalg.norm(centresOfwhale[whale,cluster]-X[dataPoint,:])
                if(euclidDist<bestEuclidianDist):
                    bestEuclidianDist=euclidDist
                    bestCluster=cluster
            dist=dist+bestEuclidianDist
            #dataPointsInCluster[whale][bestCluster].append(dataPoint)
            dataPointsInCluster[whale][bestCluster]=dataPointsInCluster[whale][bestCluster]+1
            if bestCluster==0:
                clus1=clus1+X[dataPoint]
            if bestCluster==1:
                clus2=clus2+X[dataPoint]
        if(dist<bestDist):
            bestDist=dist
            bestWhale=whale
        
        if(dataPointsInCluster[whale][0]!=0):
            centresOfwhale[whale][0]=clus1/dataPointsInCluster[whale][0]
        
        if(dataPointsInCluster[whale][1]!=0):
            centresOfwhale[whale][1]=clus1/dataPointsInCluster[whale][1]
    #shift the centroid in the centre of the datapoints inside cluster

    '''    
    print(time.time()-startTime)
    startTime=time.time();
    print("shift started")
    for whale in range(numberOfWhale):
        for cluster in range(numberOfCluster):
            numberOfPoint=len(dataPointsInCluster[whale][cluster])
            if numberOfPoint==0 :
                continue
            for feature in range(features):
                sum=0.00
                for dataPoint in dataPointsInCluster[whale][cluster]:
                    sum=sum+X[dataPoint][feature];
                sum=sum/numberOfPoint
                centresOfwhale[whale,cluster,feature]=sum'''
    
    #shift centroids using equations of WOA
#     print(time.time()-startTime)
    startTime=time.time();
#     print("woa started")
    a=2-iteration*((2.00)/iterations) #eqn 2.3
    a2=-1+iteration*((-1.00)/iterations)
    for whale in range(numberOfWhale):
        r1=random.random()
        r2=random.random()
        A=2*a*r1-a;  # Eq. (2.3) in the paper
        C=2*r2;      # Eq. (2.4) in the paper
        p=random.random()
        b=1;               #  parameters in Eq. (2.5)
        l=(a2-1)*random.random()+1;   #  parameters in Eq. (2.5)
        
        for cluster in range(numberOfCluster):
            if p<0.5 :
                if abs(A)>=1 :
                    rand_leader_index = int(math.floor((numberOfWhale-1)*random.random()+1));
                    X_rand = centresOfwhale[rand_leader_index]
                    D_X_rand=abs(C*X_rand[cluster]-centresOfwhale[whale,cluster]); # Eq. (2.7)
                    centresOfwhale[whale,cluster]=X_rand[cluster]-A*D_X_rand;      # Eq. (2.8)
                elif abs(A)<1 :
                    D_Leader=abs(C*centresOfwhale[bestWhale,cluster]-centresOfwhale[whale,cluster]); # Eq. (2.1)
                    centresOfwhale[whale,cluster]=centresOfwhale[bestWhale,cluster]-A*D_Leader;      # Eq. (2.2)
            elif p>=0.5 :
                distance2Leader=abs(centresOfwhale[bestWhale,cluster]-centresOfwhale[whale,cluster]);      # Eq. (2.5)
                centresOfwhale[whale,cluster]=distance2Leader*math.exp(b*l)*math.cos(l*2*3.14)+centresOfwhale[bestWhale,cluster];
#     print(time.time()-startTime)
    startTime=time.time();
                

In [ ]:
# from sklearn.metrics import confusion_matrix
# confusion_matrix(y,y_pred)

# from sklearn.metrics import accuracy_score
# print(accuracy_score(y,y_pred))

WOACluster=[0 for dataPoint in range(dataPoints)]
for dataPoint in range(dataPoints):
    d1=np.linalg.norm(centresOfwhale[bestWhale,0]-X[dataPoint,:])
    d2=np.linalg.norm(centresOfwhale[bestWhale,1]-X[dataPoint,:])
    d3=np.linalg.norm(centresOfwhale[bestWhale,2]-X[dataPoint,:])
#     print(d1,d2, d3)
    if d1<d2 and d1<d3 :
        WOACluster[dataPoint]=0
    elif d2<d1 and d2<d3:
        WOACluster[dataPoint]=1
    elif d3<d1 and d3<d2:
        WOACluster[dataPoint]=2
            
print(WOACluster)
# from sklearn.metrics import confusion_matrix
# confusion_matrix(y,y_pred1)

# from sklearn.metrics import accuracy_score
# print(accuracy_score(y,y_pred1))

In [ ]:
GAKMeans_Sil = metrics.silhouette_score(X, WOACluster, metric='euclidean')
GAKMeans_Sil

In [ ]:
X = data
colors = np.array(['g', 'r', 'b', 'c', 'k', 'y','royalblue', 'maroon', 'forestgreen',
                   'mediumorchid', 'tan', 'deeppink', 'olive', 'goldenrod', 'lightcyan', 'navy'])